<a href="https://colab.research.google.com/github/niikun/DL_for_health_care/blob/main/lecture2_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from google.colab import userdata
# Hugging Faceの認証トークンを環境変数に設定
os.environ["HF_TOKEN"] = userdata.get('HUGGINGFACE_TOKEN')  # トークンに置き換え
# Hugging Face Hubにログイン
login(token=os.environ["HF_TOKEN"], add_to_git_credential=True)
# トークナイザーとモデルの初期化
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B", use_fast=False)
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")


# モデルを半精度（FP16）に変換
# model.half()
# # # GPUが利用可能であれば、モデルをGPUに移動
# if torch.cuda.is_available():
#     model = model.to("cuda")
# 入力テキスト
text = "大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。"
# テキストをトークンIDにエンコード
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
# トークンIDをモデルのデバイスに移動
token_ids = token_ids.to(model.device)
# 生成プロセス
with torch.no_grad():
    output_ids = model.generate(
        token_ids,
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
# 出力をデコードして表示
output = tokenizer.decode(output_ids.tolist()[0])
print(output)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。  
- 2. 以下の文章を読んで、どのような問題を解決するために言語モデルが使われているのか説明してください。  
  - 例文:「この文章は、自然言語処理の分野でよく使われる、あるいは使われるべき、あるいは使われている、あるいは使われていない、あるいは使われない、あるいは使われないべき、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない、あるいは使われないべきではない


## Zero-shot / few-shot / CoT promptingでpromptingの効果を実感

In [ ]:
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:一貫性
[問題]:[選択肢]の中から[答え]を選んでください。
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:剣道
[問題]:[選択肢]の中から[答え]を選んでください。
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:なぎなた
[問題]:[選択肢]の中から[答え]を選んでください。
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:牡丹槍
[問題]:[選択肢]の中から[答え]を選んでください。
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:一本槍
[問題]:[選択肢]の中から[答え]を選んでください。
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:管槍
[問題]:[選択肢]の中から[答え]を選んでください


In [ ]:
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:一本槍
[問題]:一つの事柄を、複数の方法で行うこと。
[選択肢]:[一本槍, 二本槍, 三本槍, 四本槍, 五本槍]
[答え]:二本槍
[問題]:一つの事柄を、複数の方法で行うこと。
[選択肢]:[一本槍, 二本槍, 三本槍, 四本槍, 五本槍]
[答え]:三本槍
[問題]:一つの事柄を、複数の方法で行うこと。
[選択肢]:[一本槍, 二本槍, 三本槍, 四本槍, 五本槍]
[答え]:四本槍
[問題]:一つの事柄を、複数の方法で行うこと。
[選択肢]:[一本槍, 二本槍, 三本槍, 四本槍, 五本槍]
[答え]:五本槍
[問題]:一つの事柄を、複数の方法で行うこと。
[選択肢]:[一本槍, 二本槍, 三本槍, 四本槍, 五本槍]
[答え]:六本槍
[問題]:一つの


In [ ]:
# 自分のHuggingFaceアカウントと紐付ける(申請済みのアカウントでないとモデルをダウンロードできないため)、terminalでhuggingface-cli loginを実行してください。
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
# 前のセルまでに使用したGPUメモリを解放します。
# del model
del tokenizer
torch.cuda.empty_cache()
# https://huggingface.co/meta-llama
# https://huggingface.co/docs/transformers/model_doc/llama2
# https://github.com/facebookresearch/llama-recipes/tree/main
# ダウンロードには6分ほどかかります。
model_name = 'meta-llama/Llama-2-13b-hf'
tokenizer = LlamaTokenizer.from_pretrained(model_name)
# load_in_8bit=True,
model =LlamaForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

bitsandbytesは8ビット量子化をサポートするためのライブラリであり、モデルのメモリ使用量を削減するのに役立ちますが、最新バージョンが必須です。
モデルをload_in_8bit=Trueで読み込む際には、このライブラリが正しくインストールされている必要があります。
もし、ライブラリをインストールできない環境や制約がある場合は、load_in_8bit=Trueを外すことも検討できますが、その場合はメモリ消費量が増える点に注意してください。

In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: 23 - 3 * 5 = 23 - 15 = 8

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: 23 - 3 * 5 = 23 - 15 = 8

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?


In [ ]:
# Chain-of-Thought Prompting Elicits Reasoning in Large Language Models: https://arxiv.org/abs/2201.11903
prompt = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))


Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 

OpenAI

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00


In [ ]:
import os
# from dotenv import load_dotenv
# OpenAIのAPIを使うためのライブラリ
import openai
# OpenAIのtokenizerを使うためのライブラリ
import tiktoken
# API keyの取り扱いにはご注意ください。
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
openai.api_key = userdata.get('OPENAI_API_KEY')
# .envファイルにAPI keyを書いておくと、以下のように環境変数から読み込めます。
# load_dotenv()
# openai.api_key = os.environ["OPENAI_API_KEY"]
# https://platform.openai.com/docs/models/overview
openai_model_name = 'davinci-002'
openai_chat_model_name = 'gpt-3.5-turbo'

In [ ]:
# 料金が発生します
from openai import OpenAI

prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step."""

client = OpenAI(
    api_key=openai.api_key,
)

response = client.chat.completions.create(
    model=openai_chat_model_name,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
print(response.choices[0].message.content)

# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])
print('############# with space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. """

response = client.chat.completions.create(
    model=openai_chat_model_name,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

print(response.choices[0].message.content)

# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])
print('############# with 全角space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　"""

response = client.chat.completions.create(
    model=openai_chat_model_name,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

print(response.choices[0].message.content)
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])


First, Olivia spent 5 bagels x $3 = $15.
Next, we need to subtract the $15 from the original $23 Olivia had.
$23 - $15 = $8
Therefore, Olivia has $8 left after buying the bagels.
############# with space in the end #############
First, calculate how much Olivia spent on the bagels: 
5 bagels * $3 each = $15 spent on bagels 

Next, subtract the amount spent on bagels from the total amount Olivia had: 
$23 - $15 = $8

Olivia has $8 left.
############# with 全角space in the end #############
Olivia had $23 to start with. She bought 5 bagels for $3 each, so she spent 5 x $3 = $15 on bagels.
To find out how much money Olivia has left, we subtract the amount she spent on bagels from the amount she started with:
$23 - $15 = $8
Olivia has $8 left.


## Retrieval Augmented Generation
言語モデルへpromptingを行うことで、幅広いタスクに対応するイメージを掴みました。
さらなる活用方法として、言語モデルの論理思考能力をもとにしたサブタスク化を活用する方法や、言語モデル自身の重みだけでなく外部のツール・モデル・情報源を活用する方法があります。
言語モデル単体だけでタスクを行わせる場合と比べて、より難しいタスクに対応できたり、より高い精度でタスクを解かせることができると期待されます。
このような言語モデルの活用をAugmented Language Modelと呼びます。
Augmented Language Modelの一種であるRetrieval Augmented Generationについて実装し、言語モデルの活用方法についてイメージを掴みましょう。

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b")
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

### Overview
This repository provides an English-Japanese bilingual GPT-NeoX model of 3.8 billion parameters.

- Library  
The model was trained using code based on EleutherAI/gpt-neox.

- Model architecture  
A 36-layer, 2816-hidden-size transformer-based language model.
- Pre-training  
The model was trained on around 524B tokens from a mixture of the following corpora
 - Japanese CC-100  
  - Japanese C4  
  - The Pile
  - Redpajama
  - Wikipedia

https://huggingface.co/rinna/bilingual-gpt-neox-4b

In [4]:
if torch.cuda.is_available():
    model = model.to("cuda")

In [5]:
!nvidia-smi

Sun Sep  8 08:39:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              26W /  70W |  14997MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--


`!nvidia-smi` は、NVIDIA GPU（グラフィック・プロセッシング・ユニット）の状態や統計情報を確認するためのコマンドです。通常、このコマンドは、以下のような情報を表示します：  

- GPUの種類: 使用しているNVIDIA GPUのモデル名。
- メモリ使用状況: GPUの全メモリ容量と、使用中および空いているメモリの量。
- GPU使用率: 現在のGPUの処理負荷。
- 温度: GPUの温度。
- 実行中のプロセス: GPUを使用しているプロセスのリスト（例：TensorFlowやPyTorchなどの機械学習プロセス）。    

例えば、機械学習のトレーニング中にGPUリソースの利用状況を確認したり、GPUメモリが不足しているかどうかを確認するために使用されます。

このコマンドは、通常Jupyterノートブックやターミナルで実行されます。最初の ! はシェルコマンドを実行することを意味しています。


In [6]:
prompt = """
以下の質問に回答してください:
質問:　東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
回答:"""

token_ids = tokenizer.encode(prompt,add_special_tokens=False,return_tensors="pt")

- tokenizer.encode(...):  
tokenizer: これは、BERTやGPTなどのモデルで使われるトークナイザーです。トークナイザーはテキストを「トークン」という単位に分割し、それをモデルが理解できる数値（トークンID）に変換します。
- encode():   
文字列をトークン化して、数値ID（トークンID）のシーケンスに変換します。
- add_special_tokens=False:   
通常、トークナイザーはモデルに合わせて文の始まりや終わりを示す特殊なトークン（例: [CLS] や [SEP]）を自動的に追加しますが、ここではそれをオフにしています。
- return_tensors="pt":   
トークン化された結果をPyTorchのテンソル形式で返します。この場合、結果は1次元のPyTorchテンソル（行列のようなデータ形式）として返されます。

In [9]:
print(token_ids)
print(token_ids.shape)
len(prompt)

tensor([[    7,    33,  2532,  9855,   323, 16391,   431,  4621,   316,     7,
            33,  9855,   316,    33, 10529,   304, 13271, 18119,   319, 51694,
           367, 11285,  2861,  1532,  7532,   369, 22665, 22252,  4468, 44770,
           650,     7,    33, 16391,   316]])
torch.Size([1, 35])


66

- 66文字のテキストが35トークンに分割  
トークナイザーがテキストを適切に解析し、モデルが処理できる単位（トークン）に変換した結果です。テキストがトークナイズされ、35個のトークン（単語やサブワード）に変換された。
それぞれのトークンは数値ID（トークンID）として表され、これがテンソルの形で表されている。

- 不要なメモリの解放  
他のプロセスや不要なメモリを解放して、GPUメモリを確保できるようにします。例えば、コードの中で大きなテンソルやモデルが不要になったら、明示的に torch.cuda.empty_cache() を使用してキャッシュをクリアすることができます。

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

- メモリ断片化を避けるため  
この環境変数を設定することが推奨されています。これにより、PyTorchはより効率的にメモリを管理でき、メモリ割り当てエラーが回避できる可能性があります。

In [13]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [19]:
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

In [16]:
!nvidia-smi

Sun Sep  8 09:12:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              28W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

- 「モデルがGPUに乗り切らなかった」という事象  
GPUメモリには限りがあり、大きなモデルやバッチサイズがその容量を超えると、メモリ不足のエラーが発生します。今回の場合、GPUメモリにモデルや関連するデータを収めることができずにエラーが発生していたため、CPUに切り替えたことでメモリの制約が緩和され、動作したというわけです。

CPUはメモリ容量が多いものの、計算速度が遅くなることが一般的なので、GPUでのメモリ消費を抑えるための最適化も今後の選択肢として検討できます。

In [17]:
model.to('cpu')  # モデルをCPUに移動する

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(65536, 2816)
    (emb_dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-35): 36 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.1, inplace=False)
        (post_mlp_dropout): Dropout(p=0.1, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2816, out_features=8448, bias=True)
          (dense): Linear(in_features=2816, out_features=2816, bias=True)
          (attention_dropout): Dropout(p=0.1, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2816, out_features=11264, bias=True)
          (dense_4h_to_h): Linear(in_features=11264, out_features=2816, bias=Tr

In [20]:
!nvidia-smi

Sun Sep  8 09:18:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0              28W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

In [22]:
print(output_ids)
print(output_ids.shape)

tensor([[    7,    33,  2532,  9855,   323, 16391,   431,  4621,   316,     7,
            33,  9855,   316,    33, 10529,   304, 13271, 18119,   319, 51694,
           367, 11285,  2861,  1532,  7532,   369, 22665, 22252,  4468, 44770,
           650,     7,    33, 16391,   316,    33, 11285,  2861,  1532,  7532,
           369,   294,  2861,  1532,   324,  9879,   367,  3271,  5021,   514,
          7232,   354,  1362,   324,  4468,  3368,   298,     7,    33,  9855,
           316,    33, 11285,  2861,  1532,  7532,   369,   294, 22665,  2861,
          1532,   324,  4468, 44770,   650,     7,    33, 16391,   316,    33,
         11285,  2861,  1532,  7532,   369,   294,  1837,  2532,  2861,  1532,
           324,  4468,  3368,   298,     7,    33,  9855,   316,    33, 11285,
          2861,  1532,  7532,   369,   294, 22665,  2861,  1532,   324,  4468,
         44770,   650,    33,   295,   293,   297,     7,    33, 16391,   316,
            33, 11285,  2861,  1532,  7532,   369,  

In [28]:
print(output_ids.tolist()[0])
type(output_ids.tolist()[0])

[7, 33, 2532, 9855, 323, 16391, 431, 4621, 316, 7, 33, 9855, 316, 33, 10529, 304, 13271, 18119, 319, 51694, 367, 11285, 2861, 1532, 7532, 369, 22665, 22252, 4468, 44770, 650, 7, 33, 16391, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 2861, 1532, 324, 9879, 367, 3271, 5021, 514, 7232, 354, 1362, 324, 4468, 3368, 298, 7, 33, 9855, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 22665, 2861, 1532, 324, 4468, 44770, 650, 7, 33, 16391, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 1837, 2532, 2861, 1532, 324, 4468, 3368, 298, 7, 33, 9855, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 22665, 2861, 1532, 324, 4468, 44770, 650, 33, 295, 293, 297, 7, 33, 16391, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 1837, 2532, 2861, 1532, 324, 4468, 3368, 298, 7, 33, 9855, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 22665, 2861, 1532, 324, 4468, 44770, 650, 33, 295, 302, 297, 7, 33, 16391, 316, 33, 11285, 2861, 1532, 7532, 369, 294, 1837, 2532, 2861, 1532, 324, 4468, 3368, 298, 7, 33, 9855, 316, 33, 11285, 2861, 1

list

In [29]:
output = tokenizer.decode(output_ids.tolist()[0])
print(output)


 以下の質問に回答してください:
 質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか?
 回答: 大規模言語モデル講座では、言語モデルを実装する上で必要となる知識や技術を扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか?
 回答: 大規模言語モデル講座では、主に以下の言語モデルを扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか? (2)
 回答: 大規模言語モデル講座では、主に以下の言語モデルを扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか? (3)
 回答: 大規模言語モデル講座では、主に以下の言語モデルを扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか? (4)
 回答: 大規模言語モデル講座では、主に以下の言語モデルを扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか? (5)
 回答: 大規模言語モデル講座では、主に以下の言語モデルを扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか? (6)
 回答: 大規模言語モデル講座では、主に以下の言語モデルを扱います。
 質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか? (7)
 回答: 大規模言語モデル講座では、主に以下の言語


## Generation with gold passage

In [2]:
query = "東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？"
retrieved_text = """
【東大松尾研 大規模言語モデル講座開講！】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。
 """

In [3]:
prompt = f"""
以下の質問に回答してください:
質問: {query}
回答: {retrieved_text}
"""

In [5]:
token_ids = tokenizer.encode(prompt,add_special_tokens=False,return_tensors="pt")

NameError: name 'tokenizer' is not defined

In [8]:
print(token_ids)
print(token_ids.shape)
len(prompt)

tensor([[    7,    33,  2532,  9855,   323, 16391,   431,  4621,   316,     7,
            33,  9855,   316,    33, 10529,   304, 13271, 18119,   319, 51694,
           367, 11285,  2861,  1532,  7532,   369, 22665, 22252,  4468, 44770,
           650,     7,    33, 16391,   316,     7,    33,  1750, 18568, 13271,
          7301,    33, 11285,  2861,  1532,  7532, 51694,   410,  1746,     7,
            33,  7639,   393,   324,  9879,   312,  8839,   367, 43834,  7232,
         14812, 25433,  7532,   324,   299,   333,   305,   488,  7362, 51694,
           298,  3483,   402,   304, 45228,  9879,   352, 27059,    33, 12352,
           382,   333, 15078, 18963,   333, 48942,   808, 27787,  7639,   393,
         24059,  1362,   577,  1445,   926,   312, 16773,  4171, 10181,   691,
         44374,   298,  8325,  3456,   317,  1618, 18750,   298, 40332,  3749,
           317,   309, 10353, 49543,   298,     7,     7]])
torch.Size([1, 117])


218

In [9]:
model.to('cpu')

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(65536, 2816)
    (emb_dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-35): 36 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.1, inplace=False)
        (post_mlp_dropout): Dropout(p=0.1, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2816, out_features=8448, bias=True)
          (dense): Linear(in_features=2816, out_features=2816, bias=True)
          (attention_dropout): Dropout(p=0.1, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2816, out_features=11264, bias=True)
          (dense_4h_to_h): Linear(in_features=11264, out_features=2816, bias=Tr

In [10]:
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

In [11]:
print(output_ids)
print(output_ids.shape)

tensor([[    7,    33,  2532,  9855,   323, 16391,   431,  4621,   316,     7,
            33,  9855,   316,    33, 10529,   304, 13271, 18119,   319, 51694,
           367, 11285,  2861,  1532,  7532,   369, 22665, 22252,  4468, 44770,
           650,     7,    33, 16391,   316,     7,    33,  1750, 18568, 13271,
          7301,    33, 11285,  2861,  1532,  7532, 51694,   410,  1746,     7,
            33,  7639,   393,   324,  9879,   312,  8839,   367, 43834,  7232,
         14812, 25433,  7532,   324,   299,   333,   305,   488,  7362, 51694,
           298,  3483,   402,   304, 45228,  9879,   352, 27059,    33, 12352,
           382,   333, 15078, 18963,   333, 48942,   808, 27787,  7639,   393,
         24059,  1362,   577,  1445,   926,   312, 16773,  4171, 10181,   691,
         44374,   298,  8325,  3456,   317,  1618, 18750,   298, 40332,  3749,
           317,   309, 10353, 49543,   298,     7,     7,    33,  1750, 18568,
         13271,  7301,    33, 11285,  2861,  1532,  

In [13]:
print(output_ids.tolist()[0])
type(output_ids.tolist()[0])

[7, 33, 2532, 9855, 323, 16391, 431, 4621, 316, 7, 33, 9855, 316, 33, 10529, 304, 13271, 18119, 319, 51694, 367, 11285, 2861, 1532, 7532, 369, 22665, 22252, 4468, 44770, 650, 7, 33, 16391, 316, 7, 33, 1750, 18568, 13271, 7301, 33, 11285, 2861, 1532, 7532, 51694, 410, 1746, 7, 33, 7639, 393, 324, 9879, 312, 8839, 367, 43834, 7232, 14812, 25433, 7532, 324, 299, 333, 305, 488, 7362, 51694, 298, 3483, 402, 304, 45228, 9879, 352, 27059, 33, 12352, 382, 333, 15078, 18963, 333, 48942, 808, 27787, 7639, 393, 24059, 1362, 577, 1445, 926, 312, 16773, 4171, 10181, 691, 44374, 298, 8325, 3456, 317, 1618, 18750, 298, 40332, 3749, 317, 309, 10353, 49543, 298, 7, 7, 33, 1750, 18568, 13271, 7301, 33, 11285, 2861, 1532, 7532, 51694, 410, 1746, 7, 33, 7639, 393, 324, 9879, 312, 8839, 367, 43834, 7232, 14812, 25433, 7532, 324, 299, 333, 305, 488, 7362, 51694, 298, 3483, 402, 304, 45228, 9879, 352, 27059, 33, 12352, 382, 333, 15078, 18963, 333, 48942, 808, 27787, 7639, 393, 24059, 1362, 577, 1445, 926, 31

list

In [14]:
output = tokenizer.decode(output_ids.tolist()[0])
print(output)


 以下の質問に回答してください:
 質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか?
 回答:
 【東大松尾研 大規模言語モデル講座開講!】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。

 【東大松尾研 大規模言語モデル講座開講!】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。 続きを読む

 質問: LLMの実装・活用に必要な知識を体系的に学べる講座を開講するとのことですが、どのような内容ですか?
 回答:
 【東大松尾研 大規模言語モデル講座開講!】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。 続きを読む

 質問: LLMの実装・活用に必要な知識を体系的に学べる講座を開講するとのことですが、どのような内容ですか? 続きを読む

 質問: LLMの実装・活用に必要な知識を体系的に学べる講座を開講するとのことですが、どのような内容ですか? 続きを読む

 質問: LLMの実装・活用に必要な知識を体系的に学べる講座を開講するとのことですが、どのような内容ですか? 


In [7]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [13]:
import openai
from openai import OpenAI
from google.colab import userdata

prompt = f"質問: {query}"
print(prompt)
openai.api_key = userdata.get('OPENAI_API_KEY')
openai_chat_model_name = 'gpt-4o'

client = OpenAI(
    api_key=openai.api_key,
)

response = client.chat.completions.create(
    model=openai_chat_model_name,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)
print(response.choices[0].message.content)
print("############# with retrieved_text #############")
prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
"""
print(prompt)
response = client.chat.completions.create(
    model=openai_chat_model_name,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)
print(response.choices[0].message.content)

質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
東京大学の松尾研究室が開講する大規模言語モデル（Large Language Models, LLMs）に関連する講座では、以下のような内容が扱われる可能性があります。具体的な講座内容は年度やカリキュラムによって変わることがありますが、一般的なトピックは次の通りです。

1. **基礎理論**:
   - 自然言語処理（NLP）の基本概念と技術
   - 言語モデルの基本原理
   - トランスフォーマーアーキテクチャとその発展

2. **大規模言語モデルの構築**:
   - BERT、GPTシリーズ、T5などの代表的な大規模言語モデルの紹介
   - モデルのトレーニング手法、中でも自己教師あり学習や転移学習の技術
   - モデルの効率化や最適化技術

3. **応用と事例研究**:
   - 大規模言語モデルの実世界での応用例（例: 文章生成、質問応答、翻訳など）
   - 採用企業や研究機関での事例研究

4. **モデル評価と倫理問題**:
   - モデルの性能評価方法（精度、再現率、F値など）
   - モデルに伴う倫理的課題（バイアス、公平性、プライバシー問題）

5. **実践的な技術**:
   - データ収集と前処理技術
   - 実際にモデルをトレーニングするためのハンズオンセッション
   - ハードウェア・ソフトウェア環境の整備（GPU利用、分散トレーニングなど）

6. **最新の研究動向**:
   - 最新の研究論文の紹介とディスカッション
   - 今後の研究方向や未解決の問題点

7. **ツールとライブラリ**:
   - TensorFlow, PyTorchなどの機械学習ライブラリの使用方法
   - Hugging Face Transformersライブラリの活用

松尾研究室は人工知能（AI）や機械学習の分野でも著名であり、これらの講座では最新の技術動向や研究成果に基づいた深い理解と実践的なスキルを身につけることが期待されます。具体的なシラバスやコース詳細については、松尾研究室の公式サイトや大学のシラバス情報を参照するのが確実です。
############# with retrieved_text ###########

In [1]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM

前のセルまでに使用したGPUメモリを解放します。   
これでも溢れてしまう場合はRestart Kernelを試してください。  
nvidia-smiコマンドでメモリ使用量を確認できます。

In [14]:
del model
del tokenizer
torch.cuda.empty_cache()

In [4]:
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.74G [00:00<?, ?B/s]

In [5]:
!nvidia-smi

Sun Sep  8 09:51:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
if torch.cuda.is_available():
    model = model.to("cuda")

In [7]:
!nvidia-smi

Sun Sep  8 09:52:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              28W /  70W |  14997MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [46]:
import pandas as pd
df = pd.read_json("/content/drive/MyDrive/Tokyo/演習/data.json.json")
df

,url,date,title,content,text_id
0,https://weblab.t.u-tokyo.ac.jp/%e6%9d%be%e5%b0...,2023-08-31 11:28:34+00:00,松尾が登壇したG1ベンチャー2023の記事が公開されました。,松尾が登壇したG1ベンチャー2023の記事が公開されました。https://mba.glob...,0
1,https://weblab.t.u-tokyo.ac.jp/2023-08-24/,2023-08-24 13:03:42+00:00,当研究室所属の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 2023」...,当研究室博士2年の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 202...,0
2,https://weblab.t.u-tokyo.ac.jp/2023-08-24/,2023-08-24 13:03:42+00:00,当研究室所属の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 2023」...,＜古田さんより受賞コメント＞大変名誉ある賞を頂きまして、光栄に感じています。松尾先生をはじめ...,1
3,https://weblab.t.u-tokyo.ac.jp/2023-08-14/,2023-08-18 09:00:19+00:00,LLM Special Boot Campを開催しました。,このたび、深層学習・生成AIについて座学・演習を通じ学べるプログラム「LLM Special...,0
4,https://weblab.t.u-tokyo.ac.jp/2023-08-14/,2023-08-18 09:00:19+00:00,LLM Special Boot Campを開催しました。,▼共同通信https://nordot.app/1063767820357747616?c=...,1
...,...,...,...,...,...
150,https://weblab.t.u-tokyo.ac.jp/it%e3%82%a8%e3%...,2016-02-23 16:57:39+00:00,ITエンジニア本大賞・ビジネス書部門大賞受賞,松尾豊特任准教授著「人工知能は人間を超えるか ディープラーニングの先にあるもの」がITエンジ...,0
151,https://weblab.t.u-tokyo.ac.jp/%e8%87%aa%e6%b0...,2016-02-12 15:59:15+00:00,自民党「2020年以降の経済財政構想小委員会」に出席,2月10日、自民党小泉進次郎氏が事務局長を務める「2020年以降の経済財政構想小委員会」初会...,0
152,https://weblab.t.u-tokyo.ac.jp/nhk%e5%9b%bd%e9...,2016-01-15 15:54:40+00:00,NHK国際報道2016に出演しました。,松尾豊特任准教授が、1月13日特集『驚異のＡＩ技術「ディープラーニング」とは？』に出演しました。,0
153,https://weblab.t.u-tokyo.ac.jp/%e6%97%a5%e7%b5...,2016-01-08 15:45:32+00:00,日経ビジネス・特集 次代を創る100人,2015年12月28日・2016年1月4日合併号に松尾豊特任准教授の記事が掲載されました。,0


In [11]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.5 MB/s eta 0:00:00


In [16]:
from sentence_transformers import SentenceTransformer

In [17]:
model = SentenceTransformer('intfloat/multilingual-e5-large')

In [18]:
embeddings = model.encode("東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？", normalize_embeddings=True)


`normalize_embeddings=True`  
model.encode() 関数がテキストをエンコードした後に、得られた埋め込みベクトル（embeddings）を 正規化 するように指示しています。  
ここでいう正規化とは、埋め込みベクトルを L2ノルム で正規化(ベクトルの長さを1に調整すること)することを意味します。  
- なぜ正規化するのか？   
埋め込みベクトルを正規化する理由は以下の通りです：

    - コサイン類似度を計算しやすくする:  
     正規化されたベクトルを使うと、コサイン類似度がベクトルの内積と等価になります。コサイン類似度は、ベクトル間の角度を測り、ベクトルの方向のみに基づいて類似性を測るため、正規化はコサイン類似度の計算に非常に役立ちます。

    - スケーリングの一貫性:  
    埋め込みベクトルのスケールを統一することで、モデルのパフォーマンスを向上させることがあります。特に、ベクトルの長さが異なる場合、正規化しないと、その違いが類似度や距離の計算に影響を与える可能性があります

In [19]:
print(embeddings.shape)
print(embeddings)

(1024,)
[ 0.03941343  0.02392286  0.00169469 ... -0.00920513 -0.0245897
  0.01007927]


In [20]:
input_texts = [
    "passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。",
    "passage: 【学生限定・受講生募集】9月27日から開講「世界モデルと知能」の受講生を募集します！本講座では，世界モデルを軸に最新の深層学習技術を身につけることができます．深層学習の基礎を習得済みであれば，東大以外の学生も応募可能です！ 締切：9月10日（日）23:59",
    "passage: 【学生限定・締切間近！】8/25(金)より開講！東大松尾研の金融系PJチームが企画・運営する短期集中講座「金融市場取引と機械学習」の受講生を募集。金融取引に対する機械学習の活用について、理論・実装の両面から学べます。締切:8/7(月)午前10時詳細:"
    ]

In [21]:
query_embeddings = model.encode('query: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？', normalize_embeddings=True)

In [23]:
print(query_embeddings.shape)
print(query_embeddings)

(1024,)
[ 0.04601318  0.01345694  0.00154629 ... -0.00920955 -0.02119829
  0.01422186]


In [24]:
passage_embeddings = model.encode(input_texts,normalize_embeddings=True)

In [25]:
print(passage_embeddings.shape)
print(passage_embeddings)

(3, 1024)
[[ 0.01667397  0.00651591  0.00057924 ... -0.01175364 -0.02482921
   0.00795046]
 [ 0.01795118  0.00137822 -0.00598623 ... -0.01646934 -0.00543556
  -0.00059906]
 [ 0.02234265 -0.0017888  -0.01301421 ... -0.02428826 -0.02211281
   0.01296455]]


In [29]:
scores = (query_embeddings @ passage_embeddings.T) * 100
print(scores[0].tolist())

87.38448333740234


In [32]:
print(scores.argsort()[::-1])

[[1 2 0]]


In [28]:
input_texts = ["passage:" + content for content in df.content.tolist()]

In [31]:
input_texts = [
    "passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。",
    "passage: 【学生限定・受講生募集】9月27日から開講「世界モデルと知能」の受講生を募集します！本講座では，世界モデルを軸に最新の深層学習技術を身につけることができます．深層学習の基礎を習得済みであれば，東大以外の学生も応募可能です！ 締切：9月10日（日）23:59",
    "passage: 【学生限定・締切間近！】8/25(金)より開講！東大松尾研の金融系PJチームが企画・運営する短期集中講座「金融市場取引と機械学習」の受講生を募集。金融取引に対する機械学習の活用について、理論・実装の両面から学べます。締切:8/7(月)午前10時詳細:"
    ]
query_embeddings = model.encode(['query: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？'], normalize_embeddings=True)
passage_embeddings = model.encode(input_texts, normalize_embeddings=True)
print(passage_embeddings.shape)
# 類似度の計算
scores = (query_embeddings @ passage_embeddings.T) * 100
print(scores[0].tolist())
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
# 類似度の高い順のインデックスを取得
print(scores[0].argsort()[::-1])
# 一番高い類似度の文章を取得
print(input_texts[scores[0].argsort()[::-1][0]])

(3, 1024)
[87.38448333740234, 78.82355499267578, 80.8447036743164]
[0 2 1]
passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。


- scores[0].argsort()[::-1] は次のような処理を行っています：  
    - scores[0] で scores の最初の要素を取得。
    - argsort() によって、その要素を昇順に並べるためのインデックスを取得。
    - [::-1] で、そのインデックスの順番を**逆順（降順）**に変更。  
      
つまり、このコードは scores[0] の要素を降順に並べ替えた際のインデックスを取得 するという処理です。

In [35]:
scores[0].tolist()

[87.38448333740234, 78.82355499267578, 80.8447036743164]

In [36]:
input_texts

['passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。',
 'passage: 【学生限定・受講生募集】9月27日から開講「世界モデルと知能」の受講生を募集します！本講座では，世界モデルを軸に最新の深層学習技術を身につけることができます．深層学習の基礎を習得済みであれば，東大以外の学生も応募可能です！ 締切：9月10日（日）23:59',
 'passage: 【学生限定・締切間近！】8/25(金)より開講！東大松尾研の金融系PJチームが企画・運営する短期集中講座「金融市場取引と機械学習」の受講生を募集。金融取引に対する機械学習の活用について、理論・実装の両面から学べます。締切:8/7(月)午前10時詳細:']

## Retrieval Augmented Generation


In [38]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(["query:" + query],normalize_embeddings=True)
passage_embeddings = model.encode(input_texts,normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

In [39]:
scores

array([[81.690544, 75.14685 , 81.21012 ]], dtype=float32)

- コサイン類似度

In [42]:
import numpy as np
for passage_embedding in passage_embeddings:
    dot_product = query_embeddings @ passage_embedding
    norm = np.linalg.norm(query_embeddings) * np.linalg.norm(passage_embedding)
    similarity = dot_product / norm
    print(similarity.item())

0.8169054388999939
0.7514684796333313
0.8121011257171631


コサイン類似度と内積の値が同じ場合があるのは、ベクトルがすでに**正規化（L2ノルムが1に正規化されている）**されているからです。

In [52]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
passage_embeddings = model.encode(df.content.tolist(), normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100
# 上位3件を表示
print('score: ', scores[0][scores[0].argsort()[::-1][0]])
print(df.content.tolist()[scores[0].argsort()[::-1][0]])
print('score: ', scores[0][scores[0].argsort()[::-1][1]])
print(df.content.tolist()[scores[0].argsort()[::-1][1]])
print('score: ', scores[0][scores[0].argsort()[::-1][2]])
print(df.content.tolist()[scores[0].argsort()[::-1][2]])

score:  89.84386
松尾研は人工知能の研究を推進していることはよく知られていますが、ロボティクス研究にも注力していることはご存じない方も多いのではないでしょうか？今回は、そんな松尾研のロボティクス研究の目的やこれまでの経緯、活動の全体像、今後の展望について、修士時代のロボット研究の立ち上げから活躍している博士課程所属 松嶋 達也さんと、ロボットチームのアドバイザーをしている松尾研講師である岩澤 有祐さんにお話を伺いました。
score:  89.63023
ー松尾研ではなぜロボティクス研究を推進しているのですか？松嶋：松尾研がロボティクス研究を進めるのは、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えるためです。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。
score:  87.08353
松尾研では「知能を創る」というビジョンを掲げ、汎用人工知能研究に注力をしております。知能を工学的に研究するためにはNeuroAIの領域にも拡大が不可欠だと考え、山川 宏先生を中心に研究を進めております。


In [53]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(["query: "+query],normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

top_k = 2
top_k_idx = scores[0].argsort()[::-1][:top_k]

top_k_idx

array([8, 9])

In [56]:
retrieved_text = f"""
{df.content.tolist()[top_k_idx[0]]}
{df.content.tolist()[top_k_idx[1]]}
"""

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
回答:"""
print(prompt)


松尾研は人工知能の研究を推進していることはよく知られていますが、ロボティクス研究にも注力していることはご存じない方も多いのではないでしょうか？今回は、そんな松尾研のロボティクス研究の目的やこれまでの経緯、活動の全体像、今後の展望について、修士時代のロボット研究の立ち上げから活躍している博士課程所属 松嶋 達也さんと、ロボットチームのアドバイザーをしている松尾研講師である岩澤 有祐さんにお話を伺いました。
ー松尾研ではなぜロボティクス研究を推進しているのですか？松嶋：松尾研がロボティクス研究を進めるのは、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えるためです。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

上記の文章に基づいて、質問に回答してください。
質問: 松尾研ではどのようなロボティクス研究をしていますか？
回答:


In [57]:
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

token_ids = tokenizer.encode(prompt,add_special_tokens=False,return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id = tokenizer.pad_token_id,
        bos_token_id = tokenizer.bos_token_id,
        eos_token_id = tokenizer.eos_token_id
    )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [58]:
print(output_ids.shape)
print(output_ids)

torch.Size([1, 529])
tensor([[    7, 13271,  7301,   317, 40827,  3595, 36994, 16666,   317,  3226,
         16394, 28383,   319,   294,  8071, 34512,  1158,   619,  6674,   948,
         17084,   317,  1906, 12587,  1428,   894,  1283,  7879,   304,  2619,
         22256,   567,   650, 34952,   294,  9038, 13271,  7301,   304,  8071,
         34512,  1158, 23369,   354, 14960, 10648,   294,  1132,   304,  5757,
          2480,   294, 31681, 20141,   859,   294, 23955,  2627,  4704,  1158,
           304, 36192,   352, 41608, 23279,  1836,    33,  1223,  5243,    33,
         18466, 31665,   294,  4704,  6138, 24134,  3223, 13271,  7301,  5809,
           361,  1525,  2787,    33,  1271,  4308,  1276,   323,   603,   508,
           324, 42306, 10749,   358,   298,     7,   892, 13271,  7301,   369,
         13037,  8071, 34512,  1158, 36994,   468,   304, 18696,   650,  1223,
          5243,   316, 13271,  7301,   319,  8071, 34512,  1158, 20318,  1572,
           294,  1423,   862,  

In [59]:
output = tokenizer.decode(output_ids.tolist()[0])
print(output)

松尾研は人工知能の研究を推進していることはよく知られていますが、ロボティクス研究にも注力していることはご存じない方も多いのではないでしょうか?今回は、そんな松尾研のロボティクス研究の目的やこれまでの経緯、活動の全体像、今後の展望について、修士時代のロボット研究の立ち上げから活躍している博士課程所属 松嶋 達也さんと、ロボットチームのアドバイザーをしている松尾研講師である岩澤 有祐さんにお話を伺いました。
ー松尾研ではなぜロボティクス研究を推進しているのですか?松嶋:松尾研がロボティクス研究を進めるのは、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えるためです。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

上記の文章に基づいて、質問に回答してください。
質問: 松尾研ではどのようなロボティクス研究をしていますか?
回答: 松尾研では、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えています。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

松尾研では、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えています。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

松尾研では、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えています。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを